### Imports

In [1]:
import pandas as pd

url1 = "https://raw.githubusercontent.com/JoseAPBarbosa/projeto-maquinas-termicas/main/saturated_by_temperature_V1.5.csv"
url2 = "https://raw.githubusercontent.com/JoseAPBarbosa/projeto-maquinas-termicas/main/saturated_by_pressure_V1.4.csv"
url3 = "https://raw.githubusercontent.com/JoseAPBarbosa/projeto-maquinas-termicas/main/compressed_liquid_and_superheated_steam_V1.3.csv"

sat_temperature_df = pd.read_csv(url1)
sat_temperature_df.rename(columns={
    'T (°C)': 'T',
    'P (MPa)': 'P_sat',
    'Specific Volume Liquid (m^3/kg)': 'v_f',
    'Specific Volume Vapor (m^3/kg)': 'v_g',
    'Internal Energy Liquid (kJ/kg)': 'u_f',
    'Internal Energy Vapor (kJ/kg)': 'u_g',
    'Internal Energy of Vaporization (kJ/kg)': 'u_fg',
    'Enthalpy Liquid (kJ/kg)': 'h_f',
    'Enthalpy Vapor (kJ/kg)': 'h_g',
    'Enthalpy of Vaporization (kJ/kg)': 'h_fg',
    'Entropy Liquid [kJ/(kg K)]': 's_f',
    'Entropy Vapor [kJ/(kg K)]': 's_g',
    'Entropy of Vaporization [kJ/(kg K)]': 's_fg'
}, inplace=True)

sat_pressure_df = pd.read_csv(url2)
sat_pressure_df.rename(columns={
    'P (MPa)': 'P',
    'T (°C)': 'T_sat',
    'Specific Volume Liquid (m^3/kg)': 'v_f',
    'Specific Volume Vapor (m^3/kg)': 'v_g',
    'Internal Energy Liquid (kJ/kg)': 'u_f',
    'Internal Energy Vapor (kJ/kg)': 'u_g',
    'Internal Energy of Vaporization (kJ/kg)': 'u_fg',
    'Enthalpy Liquid (kJ/kg)': 'h_f',
    'Enthalpy Vapor (kJ/kg)': 'h_g',
    'Enthalpy of Vaporization (kJ/kg)': 'h_fg',
    'Entropy Liquid [kJ/(kg K)]': 's_f',
    'Entropy Vapor [kJ/(kg K)]': 's_g',
    'Entropy of Vaporization [kJ/(kg K)]': 's_fg'
}, inplace=True)

cliquid_ssteam_df = pd.read_csv(url3)
cliquid_ssteam_df.rename(columns={
    'Pressure (MPa)': 'P',
    ' Temperature (°C)': 'T',
    ' Specific Volume (m^3/kg)': 'v',
    ' Density (kg/m^3)': 'rho',
    ' Specific Internal Energy (kJ/kg)': 'u',
    ' Specific Enthalpy (kJ/kg)': 'h',
    ' Specific Entropy [kJ/(kg K)]': 's'
}, inplace=True)
cliquid_ssteam_df = cliquid_ssteam_df.apply(pd.to_numeric, errors='coerce')

### Função para obter o estado termodinâmico da água para um dada pressão e propriedade

In [2]:
def interpolated_state_at_P(df, target_P, col, target_val):
    """
    Interpolates all values in df at fixed target_P so that column `col`
    equals target_val exactly (linear interpolation).
    
    Parameters:
        df : pandas.DataFrame
        target_P : float
            Desired pressure value
        col : str
            Column name to interpolate against (e.g. 'h', 's', etc.)
        target_val : float
            Desired value in column `col`
    
    Returns:
        interpolated_row : pandas.Series
            Row with interpolated values
    """
    # Filter only rows at given pressure
    df_at_P = df[df['P'] == target_P].sort_values(by=col).reset_index(drop=True)
    
    # If the target_val is outside range, return closest row
    if target_val <= df_at_P[col].min():
        return df_at_P.iloc[0]
    if target_val >= df_at_P[col].max():
        return df_at_P.iloc[-1]

    # Find bounding rows
    lower_idx = (df_at_P[col] <= target_val).idxmax()  # first >=
    upper_idx = (df_at_P[col] >= target_val).idxmax()  # first >= again

    # Handle possible edge-case
    if lower_idx == upper_idx:
        return df_at_P.loc[lower_idx]

    row1 = df_at_P.loc[lower_idx]
    row2 = df_at_P.loc[upper_idx]

    # Linear interpolation factor
    t = (target_val - row1[col]) / (row2[col] - row1[col])

    # Interpolate every column numerically
    interpolated = {}
    for c in df.columns:
        if c == 'P':
            interpolated[c] = target_P
        elif pd.api.types.is_numeric_dtype(df[c]):
            interpolated[c] = row1[c] + t * (row2[c] - row1[c])
        else:
            # Non-numeric (like phase labels) – keep closest
            interpolated[c] = row1[c] if t < 0.5 else row2[c]

    return pd.Series(interpolated)


---

### Entrada:

In [3]:
P1 = 5.0
T1 = 240.0

m1 = 80.0

---

### Quedas de pressão

In [4]:
P2 = 1
P5 = 0.2
P6 = 0.2
P10 = 0.05

---

### Primeiro Flashing

In [5]:
state_1 = cliquid_ssteam_df[(cliquid_ssteam_df["P"] == P1) & (cliquid_ssteam_df["T"] == T1)]
h1 = state_1["h"].values[0]
s1 = state_1["s"].values[0]
x1 = 0

state_2 = sat_pressure_df[sat_pressure_df["P"] == P2]
T2 = state_2["T_sat"].values[0]
hf2 = state_2["h_f"].values[0]
hg2 = state_2["h_g"].values[0]
sf2 = state_2["s_f"].values[0]
sg2 = state_2["s_g"].values[0]

x2 = (h1 - hf2)/(hg2 - hf2)
m2 = m1

h2 = x2*hg2 + (1 - x2)*hf2
s2 = x2*sg2 + (1 - x2)*sf2

P3 = P2
T3 = T2
h3 = hf2
s3 = sf2
x3 = 0
m3 = (1-x2)*m2

P4 = P2
T4 = T2
h4 = hg2
s4 = sg2
x4 = 1
m4 = x2*m2

---

### Segundo Flashing

In [6]:
state_6 = sat_pressure_df[sat_pressure_df["P"] == P6]
T6 = state_6["T_sat"].values[0]
hf6 = state_6["h_f"].values[0]
hg6 = state_6["h_g"].values[0]
sf6 = state_6["s_f"].values[0]
sg6 = state_6["s_g"].values[0]

x6 = (h3 - hf6)/(hg6 - hf6)
m6 = m3

h6 = x6*hg6 + (1-x6)*hf6
s6 = x6*sg6 + (1-x6)*sf6

P7 = P6
T7 = T6
h7 = hf6
s7 = sf6
x7 = 0
m7 = (1-x6)*m6

P8 = P6
T8 = T6
h8 = hg6
s8 = sg6
x8 = 1
m8 = x6*m6

---

### Turbina de alta pressão


In [7]:
eta_s = 0.85
s5 = s4

state_5 = cliquid_ssteam_df[(cliquid_ssteam_df["P"] == P5) & (cliquid_ssteam_df["T"] == 120.21)]
T5 = state_5["T"].values[0]
h5f = state_5["h"].values[0]
h5g = state_5["h"].values[1]
s5f = state_5["s"].values[0]
s5g = state_5["s"].values[1]
v5f = state_5["v"].values[0]
v5g = state_5["v"].values[1]

x5 = (s5 - s5f)/(s5g - s5f)
m5 = m4

h5s = x5*h5g + (1-x5)*h5f
h5 = h4 - eta_s*(h4 - h5s)

wt1 = h5 - h4
v5 = x5*v5g + (1-x5)*v5f

---

### Misturador 5-8-9

In [8]:
P9 = P8
m9 = m5 + m8
h9 = (m5*h5 + m8*h8)/m9


state_9 = cliquid_ssteam_df[(cliquid_ssteam_df["P"] == P9) & (cliquid_ssteam_df["T"] == 120.21)]
T9 = state_9["T"].values[0]
h9f = state_9["h"].values[0]
h9g = state_9["h"].values[1]
s9f = state_9["s"].values[0]
s9g = state_9["s"].values[1]

x9 = (h9 - h9f)/(h9g - h9f)
s9 = x9*s9g + (1-x9)*s9f

---

### Turbina de baixa pressão

In [9]:
s10 = s9

state_10_s = cliquid_ssteam_df[(cliquid_ssteam_df["P"] == P10) & (cliquid_ssteam_df["T"] == 81.317)]
T10 = state_10_s["T"].values[0]
h10f = state_10_s["h"].values[0]
h10g = state_10_s["h"].values[1]
s10f = state_10_s["s"].values[0]
s10g = state_10_s["s"].values[1]

x10 = (s10 - s10f)/(s10g - s10f)
m10 = m9

h10s = x10*h10g + (1-x10)*h10f
h10 = h10s - eta_s*(h9 - h10s)

wt2 = h10 - h9

---

### Compressor 1

In [10]:
T11 = 25
P11 = P10

state_11 = cliquid_ssteam_df[(cliquid_ssteam_df["P"] == P11) & (cliquid_ssteam_df["T"] == T11)]
v11 = state_11["v"].values[0]
h11 = state_11["h"].values[0]
s11 = state_11["s"].values[0]
x11 = 0.0
m11 = m10

P12 = 0.2
wb1 = v11*(P12*10**6 - P11*10**6)*10**-3


h12 = h11 + wb1
state_12 = interpolated_state_at_P(cliquid_ssteam_df, P12, "h", h12)
T14 = state_12['T']
s12 = state_12['s']
x12 = x11
m12 = m11

### Misturador 7-12-13

In [11]:
P13 = P12
m13 = m7 + m12
h13 = (m7*h7 + m12*h12)/m13
state_13 = interpolated_state_at_P(cliquid_ssteam_df, P13, "h", h13)
T13 = state_13["T"]
v13 = state_13['v']
s13 = state_13["s"]
x13 = x12

### Compressor 2

In [12]:
P14 = 1.0
wb2 = v13*(P14*10**6 - P13*10**6)*10**-3

h14 = h13 + wb2
state_14 = interpolated_state_at_P(cliquid_ssteam_df, P14, "h", h14)
T14 = state_14['T']
s14 = state_14['s']
x14 = 0.0
m14 = m13

In [13]:
Wt = m4*wt1 + m9*wt2
Wb = m11*wb1 + m13*wb2
Wv = Wt + Wb
Ql = m10*(h10 - h11)

print(
    f"Entrada:\n"
    f"\tPoço geotérmico:\t\t{m1:.1f} kg/s\n\n"
    f"Saída:\n"
    f"\tPotência gerada:\t\t{-Wv/1000:.3f} MW\n"
    f"\tCalor liberado: \t\t{Ql/1000:.3f} MW\n\n"
    f"Emissões de CO2: Nenhuma diretamente.\n"
    )

Entrada:
	Poço geotérmico:		80.0 kg/s

Saída:
	Potência gerada:		10.186 MW
	Calor liberado: 		39.987 MW

Emissões de CO2: Nenhuma diretamente.

